# CNN Feature Etraction Model

Importing batch data

In [167]:
from pymongo import MongoClient
import pandas as pd
import tensorflow as tf
import numpy as np
from tqdm import tqdm
tqdm.pandas()

In [168]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [169]:
connection = MongoClient('localhost' , 27017)
db = connection['mydb']
collection = db['Batch_Data']

In [170]:
#Extracting the data

cursor = collection.find({}) 
batch_df = pd.DataFrame(list(cursor))

In [171]:
batch_df.head()

,_id,landmarks,label
0,6728659fbb7a61d3dd218823,"[{'face': [[0.5273651123046875, 0.255415133117...",book
1,6728659fbb7a61d3dd218824,"[{'face': [[0.24334950221882226, 0.20899586564...",book
2,6728659fbb7a61d3dd218825,"[{'face': [[0.46633575702535696, 0.22000791083...",book
3,6728659fbb7a61d3dd218826,"[{'face': [[0.47809331386904175, 0.24315066064...",book
4,6728659fbb7a61d3dd218827,"[{'face': [[0.49703465915117107, 0.09512979446...",book


In [172]:
len(batch_df['landmarks'][0])

74

In [173]:

#converting the string values into numeric values

label_encoder = LabelEncoder()
batch_df['label_encoded'] = label_encoder.fit_transform(batch_df['label'])


In [174]:
batch_df

,_id,landmarks,label,label_encoded
0,6728659fbb7a61d3dd218823,"[{'face': [[0.5273651123046875, 0.255415133117...",book,1
1,6728659fbb7a61d3dd218824,"[{'face': [[0.24334950221882226, 0.20899586564...",book,1
2,6728659fbb7a61d3dd218825,"[{'face': [[0.46633575702535696, 0.22000791083...",book,1
3,6728659fbb7a61d3dd218826,"[{'face': [[0.47809331386904175, 0.24315066064...",book,1
4,6728659fbb7a61d3dd218827,"[{'face': [[0.49703465915117107, 0.09512979446...",book,1
...,...,...,...,...
95,6728659fbb7a61d3dd218882,"[{'face': [[0.09061389265389282, 0.16436998589...",candy,2
96,6728659fbb7a61d3dd218883,"[{'face': [[0.20542802821372358, 0.22213958626...",candy,2
97,6728659fbb7a61d3dd218884,"[{'face': [[0.10087846313762443, 0.21867417570...",candy,2
98,6728659fbb7a61d3dd218885,"[{'face': [[0.5127371233913739, 0.298364802410...",candy,2


In [175]:
# # Makin sure that each landmark for video is a dictionary , i.e removig outer list

# batch_df['landmarks'] = batch_df['landmarks'].apply(lambda x : x[0] if isinstance(x,list) else x)

In [176]:
# batch_df.head()

In [177]:
len(batch_df['landmarks'][0])

74

Reshaping the Landmarks to align all the landmarks

In [178]:
def reshape_landmarks(record):

 iterations = len(record)
 
 concatenated_one_video = []

 for count in range(iterations):
 
    face_lands = np.array(record[count]['face'])
    left_hand_lands = np.array(record[count]['left_hand'])
    right_hand_lands = np.array(record[count]['right_hand'])

    #Concatinating all the landmarks to the shape (510 , 3)

    all_landmarks = np.vstack([face_lands , left_hand_lands , right_hand_lands]) # concatenated landmarks for 1 frame of the video

    concatenated_one_video.append(all_landmarks)
 
 return np.array(concatenated_one_video)

In [180]:
r1 = reshape_landmarks(batch_df['landmarks'][0])

In [182]:
r2 = reshape_landmarks(batch_df['landmarks'][1])

In [181]:
r1.shape

(74, 510, 3)

In [183]:
r2.shape

(29, 510, 3)

In [184]:
batch_df['concatenated_landmarks'] = batch_df['landmarks'].progress_apply(reshape_landmarks)

100%|██████████| 100/100 [00:01<00:00, 97.53it/s]


In [185]:
batch_df.head()

,_id,landmarks,label,label_encoded,concatenated_landmarks
0,6728659fbb7a61d3dd218823,"[{'face': [[0.5273651123046875, 0.255415133117...",book,1,"[[[0.5273651123046875, 0.2554151331174077, -0...."
1,6728659fbb7a61d3dd218824,"[{'face': [[0.24334950221882226, 0.20899586564...",book,1,"[[[0.24334950221882226, 0.20899586564690403, -..."
2,6728659fbb7a61d3dd218825,"[{'face': [[0.46633575702535696, 0.22000791083...",book,1,"[[[0.46633575702535696, 0.22000791083655372, -..."
3,6728659fbb7a61d3dd218826,"[{'face': [[0.47809331386904175, 0.24315066064...",book,1,"[[[0.47809331386904175, 0.243150660645069, -0...."
4,6728659fbb7a61d3dd218827,"[{'face': [[0.49703465915117107, 0.09512979446...",book,1,"[[[0.49703465915117107, 0.09512979446808036, -..."


In [186]:
batch_df['concatenated_landmarks'][0].shape

(74, 510, 3)

In [ ]:
one_hot_encoded_labels = tf.keras.utils.to_categorical(batch_df['label_encoded'])

In [103]:
type(one_hot_encoded_labels)

numpy.ndarray

In [104]:
one_hot_encoded_labels

array([[0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0.

Splitting the Data into Train and Test set

In [105]:
X = batch_df['concatenated_landmarks']
Y = one_hot_encoded_labels

In [106]:
# using 75 - 25 train - test split

X_train , X_test , Y_train , Y_test = train_test_split(X , Y , test_size= 0.25 , shuffle= True , random_state= 42 )

In [112]:
len(X)

100

In [111]:
X[0].shape

(510, 3)

In [109]:
Y.shape

(100, 9)

## Creating Convolutional Neural Network to Extract Feature Maps for each frames

Reshaping videos landmark data
- face - 468 landmarks
- left-hand - 21 landmarks
- right-hand - 21 landmarks
- to Pass t conv2D layer -> arranging the landmarks for each frame in the shape = (468 + 21 + 21 , 3) = (510 , 3)